Возьмем выборку из цен на Бостон и реализуем линейную регресси методом наименьших квадратов

In [388]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import scale
from copy import deepcopy
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

Вытащим данные и разобьем выборку на обучающую и тестовую

In [389]:
data = load_boston()
data_X = data.data
data_y = data.target

n = np.shape(data_X)[0]
m = np.shape(data_X)[1]
fix_data_X = np.empty((n, m + 1))
fix_data_X[:, 0] = np.ones(n)
fix_data_X[:, 1:] = np.random.randn(n, m)

X_train, X_test, y_train, y_test = train_test_split(fix_data_X, data_y, test_size=0.2)

/home/karen/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing datas

Создадим функции для получения свободных коэффициентов СЛАУ, для получения матрицы основных коэффициентов, для получения матрицы, в которой k-ый (k пробегает значения по всем переменным) столбец заменен столбцом свободных коэффициентов, а также функцию для решения самой СЛАУ методом Крамера.

In [390]:
def get_b_coeffs(data_X, data_y):
    b_coeffs = np.empty(m)
    b_coeffs = np.matmul(data_y.transpose(), data_X)
    return b_coeffs

def get_a_coeffs(data_X, data_y):
    a_coeffs = np.empty((m + 1, m + 1))
    a_coeffs = np.matmul(data_X.transpose(), data_X)
    return a_coeffs

b_s = get_b_coeffs(data_X=X_train, data_y=y_train)
a_s = get_a_coeffs(data_X=X_train, data_y=y_train)

def get_matrix_k(a_coeffs, b_coeffs, k):
    matrix = np.empty((m + 1, m + 1))
    matrix = deepcopy(a_coeffs)
    matrix[:, k] = b_coeffs
    return matrix

def solve_syst():
    A = np.empty(m + 1)
    for k in range(m + 1):
        A[k] = np.linalg.det(get_matrix_k(a_s, b_s, k)) / np.linalg.det(a_s)
    return A

coeffs = solve_syst()
y_pred = np.empty(len(y_test))
y_pred = np.matmul(X_test, coeffs)
y_pred

array([22.45376939, 23.44518487, 21.93770465, 19.83227104, 22.29583856,
       22.48548365, 23.43627561, 20.00336048, 21.52854291, 24.65892953,
       24.21954676, 24.28332321, 23.00243715, 21.58368213, 19.35228935,
       24.05321094, 24.50147695, 21.33977585, 19.26686174, 23.50904154,
       22.42337266, 20.46443165, 24.23524685, 20.84456625, 20.78131599,
       22.99778839, 21.46990467, 23.11432856, 22.03906258, 22.21217269,
       24.55914935, 22.22421642, 21.27161998, 23.78174732, 20.83336921,
       23.02203075, 22.238321  , 20.20767276, 21.74463129, 23.34701892,
       19.21740893, 20.91635932, 21.39462939, 23.52438283, 23.91843611,
       22.49185585, 23.91206937, 23.77037886, 20.7625367 , 22.29285226,
       20.96729452, 23.83079168, 22.91279107, 22.14267526, 22.38981012,
       21.57653032, 21.98543894, 22.77338209, 21.08445239, 24.59087234,
       23.98704482, 23.87967812, 24.9205008 , 20.22334193, 22.21094225,
       22.90092837, 23.22019404, 21.50576385, 20.34671644, 22.49